In [ ]:
import os
import time
import numpy as np

import matplotlib.pyplot as plt

from extra_data import open_run, RunDirectory, H5File
from extra_data.read_machinery import find_proposal

import sys
sys.path.append('../src')

from dynflatfield.constants import (
    process_dark, process_flat, process_flat_orig,
    process_flat_linear, write_constants
)
from dynflatfield.draw import plot_images

In [ ]:
propno = 2919
runno_dark = 59
runno_flat = 40

camno = 2
n_components = 20

cam_source = f"SPB_EHD_HPVX2_{camno}/CAM/CAMERA:daqOutput"

propdir = find_proposal(f"p{propno:06d}")
rundir_dark = os.path.join(propdir, f"raw/r{runno_dark:04d}")
rundir_flat = os.path.join(propdir, f"raw/r{runno_flat:04d}")

print("Proposal directory:", propdir)
print("Dark run directory:", rundir_dark)
print("Flat run directory:", rundir_flat)
print("Camera source:", cam_source)

In [ ]:
tm0 = time.monotonic()
run_dark = RunDirectory(rundir_dark)
images_dark = run_dark[cam_source, "data.image.pixels"].ndarray()
ntrain, npulse, ny, nx = images_dark.shape
tm_rd = time.monotonic() - tm0

tm0 = time.monotonic()
dark = process_dark(images_dark)
tm_cm = time.monotonic() - tm0

print(f"N image: {ntrain * npulse} (ntrain: {ntrain}, npulse: {npulse})")
print(f"Image size: {ny} x {nx} px")
print(f"Read time: {tm_rd:.2f} s, comp time: {tm_cm:.2f}")

im = plt.matshow(dark)
plt.colorbar(im)
plt.show()

In [ ]:
tm0 = time.monotonic()
run_flat = RunDirectory(rundir_flat)
images_flat = run_flat[cam_source, "data.image.pixels"].ndarray()
ntrain, npulse, ny, nx = images_flat.shape
tm_rd = time.monotonic() - tm0

tm0 = time.monotonic()
flat, components, explained_variance_ratio = process_flat_orig(
    images_flat, n_components)
tm_cm = time.monotonic() - tm0

print(f"N image: {ntrain * npulse} (ntrain: {ntrain}, npulse: {npulse})")
print(f"Image size: {ny} x {nx} px")
print(f"Read time: {tm_rd:.2f} s, comp time: {tm_cm:.2f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,5))
ax1.matshow(flat)
ax2.plot(explained_variance_ratio, 'o')
plt.show()

In [ ]:
fn = f"pca_cam{camno}_d{runno_dark}_f{runno_flat}_r{n_components}_orig.h5"
write_constants(fn, cam_source, dark, flat, components, explained_variance_ratio)

In [ ]:
plot_images(components[:20])
plt.show()
print(np.mean(components, axis=(1,2)))
print(np.std(components, axis=(1,2)))

In [ ]:
tm0 = time.monotonic()
flat, components, explained_variance_ratio = process_flat(images_flat, dark)
tm_cm = time.monotonic() - tm0

print(f"N image: {ntrain * npulse} (ntrain: {ntrain}, npulse: {npulse})")
print(f"Image size: {ny} x {nx} px")
print(f"Read time: {tm_rd:.2f} s, comp time: {tm_cm:.2f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,5))
ax1.matshow(flat)
ax2.plot(explained_variance_ratio, 'o')
plt.show()

In [ ]:
plot_images(components[:20])
plt.show()
print(np.mean(components, axis=(1,2)))
print(np.std(components, axis=(1,2)))

In [ ]:
fn = f"pca_cam{camno}_d{runno_dark}_f{runno_flat}_r{n_components}.h5"
write_constants(fn, cam_source, dark, flat, components, explained_variance_ratio)

In [ ]:
tm0 = time.monotonic()
flat, components, explained_variance_ratio = process_flat_linear(images_flat, dark, n_components)
tm_cm = time.monotonic() - tm0

print(f"N image: {ntrain * npulse} (ntrain: {ntrain}, npulse: {npulse})")
print(f"Image size: {nx} x {ny} px")
print(f"Read time: {tm_rd:.2f} s, comp time: {tm_cm:.2f}")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,5))
ax1.matshow(flat)
ax2.plot(explained_variance_ratio, 'o')
plt.show()

In [ ]:
fn = f"pca_cam{camno}_d{runno_dark}_f{runno_flat}_r{n_components}_linear.h5"
write_constants(fn, cam_source, dark, flat, components, explained_variance_ratio)

In [ ]:
plot_images(components[:20])
plt.show()
print(np.mean(components, axis=(1,2)))
print(np.std(components, axis=(1,2)))

In [ ]:
plot_images((images_flat - dark).reshape(-1, ny, nx)[5:128*20+5:128])
plt.show()